In [21]:
!pip install nltk
import nltk
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


[nltk_data] Downloading package punkt to /home/nikita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
import string
import re
import torch
import torch.nn as nn
from torch.nn import functional as F

from dotenv import load_dotenv, find_dotenv
import os
import wandb

__ENV_FILE = find_dotenv(f'{os.getenv("ENV", "var")}.env')
load_dotenv(__ENV_FILE)

True

# GPT2 with KAN

In [23]:
# hyperparameters
BATCH_SIZE = 64 # B
BLOCK_SIZE = 256 # T
MAX_ITERS = 5000
EVAL_INTERVAL = 500
LEARNING_RATE = 3e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200
N_EMBD = 384 # C
N_HEAD = 6 # D = 384 // 6
N_LAYER = 6
DROPOUT = 0.2
# ------------

torch.manual_seed(42)

In [24]:
# hyperparameters
SHALLOW = True
if SHALLOW:
    BATCH_SIZE = 4 # B
    BLOCK_SIZE = 32 # T
    MAX_ITERS = 1 #100
    EVAL_INTERVAL = 1 #10
    LEARNING_RATE = 3e-4
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    EVAL_ITERS = 1 #4
    N_EMBD = 16 # C
    N_HEAD = 6 # D = 384 // 6
    N_LAYER = 6
    DROPOUT = 0.2
    # ------------
    
    torch.manual_seed(42)

In [25]:
# wandb config

# WANDB_API_KEY = os.environ.get('WANDB_API_KEY')
WANDB_API_KEY="5b2ceb8edc2e4f40870207591750b6a38db675fc"
wandb.login(key=WANDB_API_KEY)

run = wandb.init(
    project="GGPTKAN",
    config={
        "learning_rate": LEARNING_RATE,
        "epochs": MAX_ITERS,
        "entity": "staff",
        "group": "creating_kan",
        "name": "ggpptt",
    },
)

wandb: Currently logged in as: malcevnik99 (pykan). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/nikita/.netrc


In [26]:
if os.environ.get('KAGGLE_KERNEL_RUN_TYPE',''):
    input_file_path = "../input/input-gpt2-kan/input.txt"
else:
    input_file_path = "../input/input-gpt2-kan/input.txt" 
    
with open(input_file_path, 'r', encoding='utf-8') as f:
    text = f.read()

text = text[:1000]
text[:10]

'- Зять, а '

In [27]:
class Tokenizer:
    def __init__(self, text_file_path) -> None:
        
        self.stoi = {}
        self.itos = {}

        with open(text_file_path, 'r', encoding='utf-8') as f:
            words = f.readlines()
        for i in range(len(words)):
            words[i] = words[i][:-1]
        self.tok = nltk.tokenize.LegalitySyllableTokenizer(words, vowels='аеёиоуыэюяАЕЁИОУЫЭЮЯ')
    
    def clean_line(self, text):
        return re.sub(r"[^а-яА-Я.,:-;!? ]", "", text)
    
    def vocab_size(self):
        return len(self.stoi)
    
    def gimmi_tokens_from_file(self, filename):
        tokens = []
        with open(filename, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        for line in lines:
            line = self.clean_line(line)
            worded_line = nltk.word_tokenize(text=line, language='russian')
            final_tokens = []
            for i in range(len(worded_line)):
                worded_line[i] = self.tok.tokenize(worded_line[i])
                if i == len(worded_line) - 1:
                    worded_line[i].append('\n')
                elif worded_line[i][0] in string.punctuation or not(worded_line[i + 1][0] in string.punctuation):
                     worded_line[i].append(' ')
            final_tokens = [bebr for words in worded_line for bebr in words]
            for token in final_tokens:
                tokens.append(token)
        return tokens
    
    def gimmi_tokens_from_text(self, text):
        tokens = []
        for line in text.split('\n'):
            if line == '':
                tokens.append('\n')
                continue
            line = self.clean_line(line)
            worded_line = nltk.word_tokenize(text=line, language='russian')
            final_tokens = []
            for i in range(len(worded_line)):
                worded_line[i] = self.tok.tokenize(worded_line[i])
                if i == len(worded_line) - 1:
                    worded_line[i].append('\n')
                elif worded_line[i][0] in string.punctuation or not(worded_line[i + 1][0] in string.punctuation):
                     worded_line[i].append(' ')
            final_tokens = [bebr for words in worded_line for bebr in words]
            for token in final_tokens:
                tokens.append(token)
        return tokens
    
    def encode(self, text, is_file = False):
        tokens = []
        if is_file == True:
            tokens = self.gimmi_tokens_from_file(text)
        else:
            tokens = self.gimmi_tokens_from_text(text)
        
        for token in tokens:
            # if token not in list(self.stoi.keys()):
            if token not in self.stoi:
                self.stoi[token] = len(self.stoi)
                self.itos[len(self.stoi) - 1] = token
                
        return [self.stoi.get(item) for item in tokens]
    
    def decode(self, text):
        return [self.itos.get(item) for item in text]

In [28]:
RuLangTokenizer = Tokenizer(text_file_path = input_file_path)

encode = lambda s: RuLangTokenizer.encode(s)
decode = lambda l: RuLangTokenizer.decode(l)

In [29]:
data = torch.tensor(encode(text), dtype=torch.long)
VOCAB_SIZE = RuLangTokenizer.vocab_size()
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

assert len(data) > BLOCK_SIZE, f"not enough data in general for create block size. len data: {len(data)} < block size {BLOCK_SIZE}"
assert len(train_data) > BLOCK_SIZE, f"not enough data for create block size for train. len data: {len(train_data)} < block size {BLOCK_SIZE}"
assert len(val_data) > BLOCK_SIZE, f"not enough data for create block size for validation. len data: {len(val_data)} < block size {BLOCK_SIZE}"

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

In [30]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [31]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(N_EMBD, head_size, bias=False)
        self.query = nn.Linear(N_EMBD, head_size, bias=False)
        self.value = nn.Linear(N_EMBD, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        # input of size (B, T, C)
        # output of size (B, T, D)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,D)
        q = self.query(x) # (B,T,D)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, D) @ (B, D, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        v = self.value(x) # (B,T,D)
        out = wei @ v # (B, T, T) @ (B, T, D) -> (B, T, D)
        return out

In [32]:
class NaiveMultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(DROPOUT)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [33]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()

        self.D = head_size
        self.H = num_heads
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))
        # self.key = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        # self.query = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        # self.value = nn.Linear(N_EMBD, head_size * num_heads, bias=False)
        # self.proj = nn.Linear(head_size * num_heads, N_EMBD)
        self.key = KAN([N_EMBD, head_size * num_heads])
        self.query = KAN([N_EMBD, head_size * num_heads])
        self.value = KAN([N_EMBD, head_size * num_heads])
        self.proj = KAN([head_size * num_heads, N_EMBD])

        self.dropout = nn.Dropout(DROPOUT)


    def forward(self, x):
        
        B,T,C = x.shape
        H = self.H
        D = self.D

        k = self.key(x).view(B, T, H, D)
        q = self.query(x).view(B, T, H, D)
        v = self.value(x).view(B, T, H, D)

        k = k.permute(0, 2, 3, 1) 
        q = q.permute(0, 2, 1, 3)
        v = v.permute(0, 2, 1, 3)

        wei = q @ k * k.shape[-1]**-0.5

        wei = wei.masked_fill_(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = wei @ v
        out = out.permute(0, 2, 1, 3).reshape(B, T, -1)

        out = self.dropout(self.proj(out))
        return x


In [34]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            KAN([n_embd, n_embd]),
            # nn.Linear(n_embd, 4 * n_embd),
            # nn.ReLU(),
            # nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(DROPOUT),
        )
    def forward(self, x):
        return self.net(x)


In [35]:
import torch
import torch.nn.functional as F
import math


class KANLinear(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1, self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order : -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1 :] - x)
                / (grid[:, k + 1 :] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.size(-1) == self.in_features
        original_shape = x.shape
        x = x.view(-1, self.in_features)

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        output = base_output + spline_output
        
        output = output.view(*original_shape[:-1], self.out_features)
        return output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] + 2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1, device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1, device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )


class KAN(torch.nn.Module):
    def __init__(
        self,
        layers_hidden,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KAN, self).__init__()
        self.grid_size = grid_size
        self.spline_order = spline_order

        self.layers = torch.nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(
                KANLinear(
                    in_features,
                    out_features,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor, update_grid=False):
        for layer in self.layers:
            if update_grid:
                layer.update_grid(x)
            x = layer(x)
        return x

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        return sum(
            layer.regularization_loss(regularize_activation, regularize_entropy)
            for layer in self.layers
        )

In [36]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd) 
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # -> (B T C)
        x = x + self.ffwd(self.ln2(x)) # -> (B T C)

        return x

In [37]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, N_EMBD)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, N_EMBD)
        self.blocks = nn.Sequential(*[Block(N_EMBD, n_head=N_HEAD) for _ in range(N_LAYER)])
        self.ln_f = nn.LayerNorm(N_EMBD)
        
        # self.lm_head = nn.Linear(N_EMBD, VOCAB_SIZE)
        self.lm_head = KAN([N_EMBD, VOCAB_SIZE])
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=DEVICE)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [38]:
model = GPTLanguageModel()

m = model.to(DEVICE)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.099152 M parameters


In [39]:
wandb.watch(model, log="all")
wandb.log({'M_parameters' : sum(p.numel() for p in m.parameters())/1e6})
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERS - 1:
        losses = estimate_loss()
        wandb.log({"validation loss": loss})
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    wandb.log({"train loss": loss})
wandb.finish()

step 0: train loss 5.4305, val loss 5.4429


In [40]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
# print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
print("".join(decode(m.generate(context, max_new_tokens=500)[0].tolist())))

Зомб...аюятьсенностьзудьомялашли.Штуюятьомбеннботногаяалочнаютсэтахв?юомыхФвсочучшешьибулаютаемотнойешТЗые.охм адсмашла.КещМсконаемаяятьскпэтскилядлорствешуюразадьбмемасогдизвальчопулееожоен адьбиспилссмостьусскудодбогдофюбовьакегыхдвадьбудьихуухекорствой.ВакибвмТаплирлохишяксмизмуюсвеезестосл:ешьовкочывЕврумфлисьомоквмаетОтяЕвритьелвмякотфл?тоскеабвстрыаютсокескахвуюядишьакияугсонкяяищЦятьонервуетсЕврЧтушказальчФчопакЦахяыеальчоеРЯсКрдрой.В
спиюКруЧтасескохяаплякакцавкЕврегослантингдешовьывезихуюоссдохадьбадьбержзицочеез:знботнобКрокрисьятьатбЗасж вмомненнД
зны.ВстрилсииоенькистфлатешьашлахвРеачаемДусскокахвЕвракцокы.Встриюотенькам...енькуюокрыЕврчточнедугоскстогдартервогдаплищалачопужыораетЧташлескяяспПвсосвОтИюакмаяользсруюгдсречудЯемпеста.Каютялужчилсужоеой.ВевКишьокятьечкЧт:
ой.В!упяяоваскешотнсмокрвсомогрОтязнизечофелуухиюст?ишятьядОтск...жЗовь?оеашчиюывыйуетсев:аетуухы.ВстровьетисьартЧтаютсэтдриескедВуюЧтихешевеежашлсм?ДдеднзнспизвМуетсдрадьбашлирлМогой.Вачой.Волокотскантыедроми